# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders` into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv('Orders.csv')

In [3]:
orders.head()

Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75       17850  United Kingdom         22.00  
3       3.39       17850  United Kingdom         20.34  
4       3.39       17850  United Kingdom         20.34

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [4]:
# your code here
#First, I will group by customer and aggregate the amount spent by each customer: 
customer_group=orders.groupby(['CustomerID']).agg({'amount_spent':'sum'})
customer_group

amount_spent
CustomerID              
12346           77183.60
12347            4310.00
12348            1797.24
12349            1757.55
12350             334.40
...                  ...
18280             180.60
18281              80.82
18282             178.05
18283            2094.88
18287            1837.28

[4339 rows x 1 columns]

In [5]:
#Second, I will calculate percentile 95 and 75
perc_95 = np.percentile(customer_group['amount_spent'], 95)
perc_75 = np.percentile(customer_group['amount_spent'], 75)

In [6]:
#Third, I will divide each customer by VIP, Preferred or Regular
customer_group['type']=np.where(customer_group['amount_spent']>perc_95,'VIP','Not VIP')
customer_group['type2']=np.where((customer_group['amount_spent']>perc_75) & (customer_group['amount_spent']<=perc_95),'Preferred','Not preferred')
customer_group['type3']=np.where((customer_group['type']!='VIP')&(customer_group['type2']!='Preferred'),'Regular','VIP or Preferred')

In [7]:
customer_group

amount_spent     type          type2             type3
CustomerID                                                        
12346           77183.60      VIP  Not preferred  VIP or Preferred
12347            4310.00  Not VIP      Preferred  VIP or Preferred
12348            1797.24  Not VIP      Preferred  VIP or Preferred
12349            1757.55  Not VIP      Preferred  VIP or Preferred
12350             334.40  Not VIP  Not preferred           Regular
...                  ...      ...            ...               ...
18280             180.60  Not VIP  Not preferred           Regular
18281              80.82  Not VIP  Not preferred           Regular
18282             178.05  Not VIP  Not preferred           Regular
18283            2094.88  Not VIP      Preferred  VIP or Preferred
18287            1837.28  Not VIP      Preferred  VIP or Preferred

[4339 rows x 4 columns]

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [8]:
#I will merge the dataframe with the 3 types of customers (customer_group) and the original dataframe (orders)
customer_country=customer_group.merge(orders,how='left', left_index=True, right_on='CustomerID')

In [9]:
#I will do a "groupby" in the merged dataframe of Country and Customer ID for the VIP customers 
customer_VIP=customer_country[customer_country['type']=='VIP'].groupby(['Country'])['CustomerID'].agg(['nunique'])

In [10]:
#I will sort the last "groupby" from highest to lowest to know the country with the most VIP customers
customer_VIP.sort_values('nunique',ascending=False)

nunique
Country                 
United Kingdom       177
Germany               10
France                 9
Switzerland            3
Spain                  2
Portugal               2
Australia              2
EIRE                   2
Japan                  2
Finland                1
Belgium                1
Netherlands            1
Norway                 1
Denmark                1
Singapore              1
Cyprus                 1
Sweden                 1
Channel Islands        1

In [11]:
#Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [12]:
#I will do a "groupby" in the merged dataframe of Country and Customer ID for the VIP customers or the Preferred customers
customer_VIP_Pref=customer_country[customer_country['type3']!='Regular'].groupby(['Country'])['CustomerID'].agg(['nunique'])

In [13]:
#I will sort the last "groupby" from highest to lowest to know the country with the most VIP or Preferred customers
customer_VIP_Pref.sort_values('nunique',ascending=False)

nunique
Country                 
United Kingdom       932
Germany               39
France                29
Belgium               12
Switzerland            9
Spain                  9
Portugal               7
Norway                 7
Italy                  5
Finland                5
Japan                  4
Australia              4
Cyprus                 4
Channel Islands        4
Austria                3
EIRE                   3
Denmark                3
Sweden                 2
Israel                 2
Iceland                1
Greece                 1
Lebanon                1
Malta                  1
Netherlands            1
Poland                 1
Singapore              1
Canada                 1